<a href="https://colab.research.google.com/github/Qiaochu-Zhang/C-_Grades_with_Hash_Table/blob/main/efficientNet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install torch torchvision
!pip install opencv-python

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import os
from PIL import Image
import numpy as np
import random
import timm  # PyTorch Image Models库

# 检查是否有 GPU 可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# 自定义数据集类，直接读取预处理后的图片
class PreprocessedDataset(Dataset):
    def __init__(self, image_folder, label, transform=None):
        self.image_paths = [
            os.path.join(image_folder, img)
            for img in os.listdir(image_folder)
            if img.lower().endswith(('.jpg', '.jpeg', '.png'))
        ]
        self.labels = [label] * len(self.image_paths)
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # 直接读取预处理后的图片
        image = Image.open(self.image_paths[idx]).convert('L')  # 单通道灰度图像
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

# 定义数据转换
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# 定义预处理后图片的保存目录
processed_real_train_save_dir = '/content/drive/MyDrive/processed_real_train'
processed_real_test_save_dir = '/content/drive/MyDrive/processed_real_test'
processed_fake_train_save_dir = '/content/drive/MyDrive/processed_fake_train'
processed_fake_test_save_dir = '/content/drive/MyDrive/processed_fake_test'

# 加载训练数据集
real_train_dataset = PreprocessedDataset(processed_real_train_save_dir, label=0, transform=train_transform)
fake_train_dataset = PreprocessedDataset(processed_fake_train_save_dir, label=1, transform=train_transform)
train_dataset = ConcatDataset([real_train_dataset, fake_train_dataset])

# 加载测试数据集
real_test_dataset = PreprocessedDataset(processed_real_test_save_dir, label=0, transform=test_transform)
fake_test_dataset = PreprocessedDataset(processed_fake_test_save_dir, label=1, transform=test_transform)
test_dataset = ConcatDataset([real_test_dataset, fake_test_dataset])

print(f"Training set size: {len(train_dataset)}")
print(f"Test set size: {len(test_dataset)}")

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# 初始化 EfficientNet 模型
def initialize_efficientnet():
    model = timm.create_model('efficientnet_b0', pretrained=True)
    # 修改第一层卷积层为单通道输入
    original_conv_stem = model.conv_stem
    model.conv_stem = nn.Conv2d(
        in_channels=1,
        out_channels=original_conv_stem.out_channels,
        kernel_size=original_conv_stem.kernel_size,
        stride=original_conv_stem.stride,
        padding=original_conv_stem.padding,
        bias=original_conv_stem.bias is not None
    )
    with torch.no_grad():
        model.conv_stem.weight = nn.Parameter(
            original_conv_stem.weight.sum(dim=1, keepdim=True)
        )
    # 修改分类层为 2 类输出
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, 2)
    return model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
model = initialize_efficientnet()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# 学习率调度器
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=2, verbose=True)

# 训练模型
num_epochs = 10
best_val_loss = float('inf')
early_stop_patience = 5
early_stop_counter = 0

for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    model.train()
    running_loss = 0.0

    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f'Training Loss: {avg_train_loss:.4f}')

    # 在测试集上评估模型
    model.eval()
    val_loss = 0.0
    correct, total = 0, 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            loss = criterion(outputs, target)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    avg_val_loss = val_loss / len(test_loader)
    val_accuracy = 100 * correct / total
    print(f'Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')

    # 学习率调度器步进
    scheduler.step(avg_val_loss)

    # 早停策略
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stop_counter = 0
        # 保存最佳模型权重
        torch.save(model.state_dict(), 'best_model_efficientnet.pth')
    else:
        early_stop_counter += 1
        if early_stop_counter >= early_stop_patience:
            print("Early stopping triggered")
            break

# 加载最佳模型参数
model.load_state_dict(torch.load('best_model_efficientnet.pth'))

# 在测试集上最终评估模型
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

test_accuracy = 100 * correct / total
print(f'\nFinal Test Accuracy: {test_accuracy:.2f}%')


Using device: cuda
Training set size: 4917
Test set size: 1222


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Training Loss: 0.4044
Validation Loss: 0.3309, Accuracy: 85.60%
Epoch 2/10
Training Loss: 0.2587
Validation Loss: 0.2229, Accuracy: 89.61%
Epoch 3/10
Training Loss: 0.2059
Validation Loss: 0.1851, Accuracy: 93.54%
Epoch 4/10
Training Loss: 0.1822
Validation Loss: 0.8977, Accuracy: 77.82%
Epoch 5/10
Training Loss: 0.1684
Validation Loss: 0.2283, Accuracy: 90.67%
Epoch 6/10
Training Loss: 0.1467
Validation Loss: 0.1448, Accuracy: 94.60%
Epoch 7/10
Training Loss: 0.1501
Validation Loss: 0.4752, Accuracy: 83.63%
Epoch 8/10
Training Loss: 0.1396
Validation Loss: 0.5185, Accuracy: 82.90%
Epoch 9/10
Training Loss: 0.1258
Validation Loss: 0.2239, Accuracy: 91.49%
Epoch 10/10
Training Loss: 0.0943
Validation Loss: 0.0996, Accuracy: 96.64%


<ipython-input-4-120459005392>:170: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_efficientnet.pth'))



Final Test Accuracy: 96.64%


In [5]:
# 假设您的模型实例名为 model
torch.save(model.state_dict(), '/content/drive/MyDrive/best_model_epoch10_eff_sec2.pth')


In [6]:
from google.colab import files
files.download('/content/drive/MyDrive/best_model_epoch10_eff_sec2.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>